In [1]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [2]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [3]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: None


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [4]:
data = data.with_columns(
    pl.col("timestamp")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f", strict=False)
    .alias("timestamp")
)

/tmp/ipykernel_49379/421030682.py:2: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("timestamp")


In [5]:
data.collect()

car_id,timestamp,latitude,longitude,notification
str,datetime[μs],f64,f64,str
"""car_0""",null,24.533157,-108.595081,null
"""car_0""",2023-01-01 00:01:33.000991,24.533157,-108.595081,"""low_fuel"""
"""car_0""",2023-01-01 00:03:12.000208,24.533157,-108.595081,"""tire_pressure"""
"""car_0""",2023-01-01 00:03:12.000612,24.533157,-108.595081,null
"""car_0""",null,24.542875,-108.596283,"""engine_check"""
…,…,…,…,…
"""car_999""",2023-01-07 23:07:09.000601,28.023287,-96.601204,"""engine_check"""
"""car_999""",2023-01-07 23:02:42.000997,28.023287,-96.601204,"""tire_pressure"""
"""car_999""",2023-01-07 23:06:26.000857,28.023287,-96.601204,"""engine_check"""


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [8]:
data = data.with_columns(
    pl.col("timestamp").diff().over("car_id").alias("notification_time")
 )
data.collect()

car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]
"""car_0""",null,24.533157,-108.595081,null,0.0,null
"""car_0""",2023-01-01 00:01:33.000991,24.533157,-108.595081,"""low_fuel""",0.0,null
"""car_0""",2023-01-01 00:03:12.000208,24.533157,-108.595081,"""tire_pressure""",0.0,1m 38s 999217µs
"""car_0""",2023-01-01 00:03:12.000612,24.533157,-108.595081,null,2.174802,404µs
"""car_0""",null,24.542875,-108.596283,"""engine_check""",0.0,null
…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:07:09.000601,28.023287,-96.601204,"""engine_check""",0.0,-3m -58s -999538µs
"""car_999""",2023-01-07 23:02:42.000997,28.023287,-96.601204,"""tire_pressure""",0.0,-4m -26s -999604µs
"""car_999""",2023-01-07 23:06:26.000857,28.023287,-96.601204,"""engine_check""",0.0,3m 43s 999860µs


2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [6]:
# Radius of Earth in kilometers
R = 6371.0

# Convert latitude and longitude columns to radians
data = data.with_columns([
    (pl.col("latitude") * np.pi / 180).alias("lat_rad"),
    (pl.col("longitude") * np.pi / 180).alias("lon_rad"),
])

# Calculate Haversine distance with correct usage of sin, cos, and arcsin as chained methods
data = data.with_columns([
    (2 * R * (
        (pl.col("lat_rad").shift(-1) - pl.col("lat_rad")).sin().pow(2) +
        (pl.col("lat_rad").cos() * pl.col("lat_rad").shift(-1).cos() *
         (pl.col("lon_rad").shift(-1) - pl.col("lon_rad")).sin().pow(2))
    ).sqrt().arcsin()).over("car_id").alias("distance")
])

# Drop the intermediate columns if no longer needed
data = data.drop(["lat_rad", "lon_rad"])

# Collect and display the results
data_df = data.collect()
data_df

car_id,timestamp,latitude,longitude,notification,distance
str,datetime[μs],f64,f64,str,f64
"""car_0""",null,24.533157,-108.595081,null,0.0
"""car_0""",2023-01-01 00:01:33.000991,24.533157,-108.595081,"""low_fuel""",0.0
"""car_0""",2023-01-01 00:03:12.000208,24.533157,-108.595081,"""tire_pressure""",0.0
"""car_0""",2023-01-01 00:03:12.000612,24.533157,-108.595081,null,2.174802
"""car_0""",null,24.542875,-108.596283,"""engine_check""",0.0
…,…,…,…,…,…
"""car_999""",2023-01-07 23:07:09.000601,28.023287,-96.601204,"""engine_check""",0.0
"""car_999""",2023-01-07 23:02:42.000997,28.023287,-96.601204,"""tire_pressure""",0.0
"""car_999""",2023-01-07 23:06:26.000857,28.023287,-96.601204,"""engine_check""",0.0


In [9]:
data.filter(pl.col("distance") > 0).collect()

car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]
"""car_0""",2023-01-01 00:03:12.000612,24.533157,-108.595081,null,2.174802,404µs
"""car_0""",2023-01-01 00:43:09.000533,24.542875,-108.596283,"""low_fuel""",1.628738,7m 9s 999538µs
"""car_0""",2023-01-01 01:01:40.000805,24.540011,-108.588873,"""tire_pressure""",1.482681,-11m -38s -999623µs
"""car_0""",2023-01-01 01:37:50.000369,24.538638,-108.581701,"""tire_pressure""",1.848402,-1m -52s -999635µs
"""car_0""",2023-01-01 02:03:45.000365,24.530336,-108.582129,null,0.62616,-2m -45s -349µs
…,…,…,…,…,…,…
"""car_999""",2023-01-07 21:07:06.000336,28.016826,-96.609589,null,2.569785,-13m -29s -300µs
"""car_999""",2023-01-07 21:35:03.000770,28.026443,-96.602332,"""engine_check""",2.328463,3m 11s 468µs
"""car_999""",2023-01-07 22:11:13.000595,28.035543,-96.608197,"""tire_pressure""",1.407846,10m 21s 230µs


3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [10]:
data = data.with_columns([
    pl.col("timestamp").fill_null(strategy="forward").cast(pl.Datetime)
])
null_timestamps = data.filter(pl.col("timestamp").is_null()).collect()
print("Rows with null timestamps:", null_timestamps.shape[0])

Rows with null timestamps: 1


In [11]:
null_timestamps

car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]
"""car_0""",null,24.533157,-108.595081,null,0.0,null


In [12]:
data = data.with_columns([
    pl.col("timestamp").fill_null(strategy="backward").cast(pl.Datetime)
])

In [13]:
null_timestamps = data.filter(pl.col("timestamp").is_null()).collect()
print("Rows with null timestamps:", null_timestamps.shape[0])
null_timestamps

Rows with null timestamps: 0


car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]


In [14]:
data = data.with_columns([
    pl.col("notification_time").fill_null(pl.duration(microseconds=0)),
    pl.col("distance").fill_null(0.0)
])
data.collect()

car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]
"""car_0""",2023-01-01 00:01:33.000991,24.533157,-108.595081,null,0.0,0µs
"""car_0""",2023-01-01 00:01:33.000991,24.533157,-108.595081,"""low_fuel""",0.0,0µs
"""car_0""",2023-01-01 00:03:12.000208,24.533157,-108.595081,"""tire_pressure""",0.0,1m 38s 999217µs
"""car_0""",2023-01-01 00:03:12.000612,24.533157,-108.595081,null,2.174802,404µs
"""car_0""",2023-01-01 00:03:12.000612,24.542875,-108.596283,"""engine_check""",0.0,0µs
…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:07:09.000601,28.023287,-96.601204,"""engine_check""",0.0,-3m -58s -999538µs
"""car_999""",2023-01-07 23:02:42.000997,28.023287,-96.601204,"""tire_pressure""",0.0,-4m -26s -999604µs
"""car_999""",2023-01-07 23:06:26.000857,28.023287,-96.601204,"""engine_check""",0.0,3m 43s 999860µs


In [15]:
data.filter(pl.col("notification_time").is_null()).collect()

car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]


In [16]:
data.filter(pl.col("distance").is_null()).collect()

car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]


In [17]:
data.sort(["car_id", "timestamp"])
data.collect()

car_id,timestamp,latitude,longitude,notification,distance,notification_time
str,datetime[μs],f64,f64,str,f64,duration[μs]
"""car_0""",2023-01-01 00:01:33.000991,24.533157,-108.595081,null,0.0,0µs
"""car_0""",2023-01-01 00:01:33.000991,24.533157,-108.595081,"""low_fuel""",0.0,0µs
"""car_0""",2023-01-01 00:03:12.000208,24.533157,-108.595081,"""tire_pressure""",0.0,1m 38s 999217µs
"""car_0""",2023-01-01 00:03:12.000612,24.533157,-108.595081,null,2.174802,404µs
"""car_0""",2023-01-01 00:03:12.000612,24.542875,-108.596283,"""engine_check""",0.0,0µs
…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:07:09.000601,28.023287,-96.601204,"""engine_check""",0.0,-3m -58s -999538µs
"""car_999""",2023-01-07 23:02:42.000997,28.023287,-96.601204,"""tire_pressure""",0.0,-4m -26s -999604µs
"""car_999""",2023-01-07 23:06:26.000857,28.023287,-96.601204,"""engine_check""",0.0,3m 43s 999860µs


In [18]:
data = data.with_columns([
    pl.col("timestamp").fill_null(strategy="forward").cast(pl.Datetime),
    pl.col("notification_time").fill_null(pl.duration(microseconds=0)),
    pl.col("distance").fill_null(0.0).cast(pl.Float64)
]).sort(["car_id", "timestamp"]).collect() 

In [19]:
data = data.lazy()

In [20]:
x_minutes = 5
data_aggregated = (data
    .group_by_dynamic(
        index_column="timestamp",
        every=f"{x_minutes}m",
        by="car_id"
    )
    .agg([
        pl.col("notification_time").mean().alias("mean_notification_time"),
        pl.col("notification_time").median().alias("median_notification_time"),
        pl.col("notification_time").var().alias("var_notification_time"),
        pl.col("notification_time").max().alias("max_notification_time"),
        pl.col("notification_time").min().alias("min_notification_time"),
        pl.col("distance").mean().alias("mean_distance"),
        pl.col("distance").median().alias("median_distance"),
        pl.col("distance").var().alias("var_distance"),
        pl.col("distance").max().alias("max_distance"),
        pl.col("distance").min().alias("min_distance"),
    ])
)

result = data_aggregated.collect()
result

/tmp/ipykernel_49379/2324486689.py:2: DeprecationWarning: The argument `by` for `LazyFrame.group_by_dynamic` is deprecated. It has been renamed to `group_by`.
  data_aggregated = (data


car_id,timestamp,mean_notification_time,median_notification_time,var_notification_time,max_notification_time,min_notification_time,mean_distance,median_distance,var_distance,max_distance,min_distance
str,datetime[μs],duration[μs],duration[μs],duration[μs],duration[μs],duration[μs],f64,f64,f64,f64,f64
"""car_0""",2023-01-01 00:00:00,19s 799924µs,0µs,22687d 2h 16m 33s 786894µs,1m 38s 999217µs,0µs,0.43496,0.0,0.945953,2.174802,0.0
"""car_0""",2023-01-01 00:30:00,-9m -9s -571µs,-9m -9s -571µs,null,-9m -9s -571µs,-9m -9s -571µs,0.0,0.0,null,0.0,0.0
"""car_0""",2023-01-01 00:35:00,2m 15s 500479µs,2m 15s 500479µs,1940050d 11h 14m 21s 29056µs,7m 5s 359µs,-2m -33s -999400µs,0.0,0.0,0.0,0.0,0.0
"""car_0""",2023-01-01 00:40:00,-2m -42s -250362µs,-2m -54s -340µs,2835898d 3h 54m 35s 915264µs,7m 9s 999538µs,-12m -11s -306µs,0.407184,0.0,0.663197,1.628738,0.0
"""car_0""",2023-01-01 00:45:00,3m 43s 500054µs,3m 43s 500054µs,1156302d 15h 24m 36s 5824µs,7m 27s 108µs,0µs,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…
"""car_999""",2023-01-07 22:45:00,6m 326µs,6m 326µs,null,6m 326µs,6m 326µs,0.0,0.0,null,0.0,0.0
"""car_999""",2023-01-07 23:00:00,6m 24s 95µs,49s 999983µs,8712854d 19h 54m 22s 68992µs,22m 48s 999908µs,-4m -26s -999604µs,0.0,0.0,0.0,0.0,0.0
"""car_999""",2023-01-07 23:05:00,1m 29s 666762µs,3m 43s 999860µs,948103d 15h 44m 41s 103408µs,4m 43s 999965µs,-3m -58s -999538µs,0.0,0.0,0.0,0.0,0.0
